Lightning en 15 minutos, definiendo un LightningModule:

In [27]:
import os #El sistema
from torch import optim, nn, utils, Tensor #Para crear las redes neuronales, los optimizadores, manejar los datos y hacer operaciones complejas con tensores 
from torchvision.datasets import MNIST #Este es un conjunto de datos de ejemplo
from torchvision.transforms import ToTensor #Esto nos va a ayudar a transformar 
import lightning as L #Lightning ahora sí 
import torch.nn.functional as F


In [5]:
# define any number of nn.Modules (or use your current ones)
#Estos son los componentes del modelo: 

encoder = nn.Sequential(nn.Linear(28 * 28, 64), nn.ReLU(), nn.Linear(64, 3)) #Comprimir las dimensiones hasta dimensión 3, también está la función de activación ReLu
decoder = nn.Sequential(nn.Linear(3, 64), nn.ReLU(), nn.Linear(64, 28 * 28))  #Descomprimir de dimensión 3 hasta 28*28


# define the LightningModule, estamos aquí definiendo el modulo que es heredado de Lightning
#Es decir, tendrá los mismos atributos y acciones que lightning
class LitAutoEncoder(L.LightningModule):
    def __init__(self, encoder, decoder):
        super().__init__() #Para iniciar la clase padre 
        self.encoder = encoder #Definimos sus propios encoder
        self.decoder = decoder #Definimos sus propios decoder 

    def training_step(self, batch, batch_idx):
        # El loop de training
        x, _ = batch #Imagen con su etiqueta 
        x = x.view(x.size(0), -1) #Pasar la imagen a un vector
        z = self.encoder(x) #Comprimir
        x_hat = self.decoder(z) #Descomprimir 
        loss = nn.functional.mse_loss(x_hat, x) #Comparación de esta reconstrucción
        # Logging to TensorBoard (if installed) by default
        self.log("train_loss", loss) #Guardar el valor de pérdida
        return loss #Regresa este valor de pérdida

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=1e-3) #Seleccionamos un optmimizador 
        return optimizer


# init the autoencoder
autoencoder = LitAutoEncoder(encoder, decoder) #Se crea un objeto de la clase definida anteriormente 

In [69]:
# setup data
dataset = MNIST(os.getcwd(), download=True, transform=ToTensor()) #Descargamos la base de datos, las imagenes y las transformamos en tensores
train_loader = utils.data.DataLoader(dataset) # Esto clasifica las imagenes de la base en batches 

In [9]:
#El entrenador de lightning
trainer = L.Trainer(limit_train_batches=100, max_epochs=1) #Trabajara con batches de 100 imagenes y sólo lo hará una vez
trainer.fit(model=autoencoder, train_dataloaders=train_loader) #Esto hará que el entrenamiento se haga con lo que definimos antes

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name    | Type       | Params | Mode 
-----------------------------------------------
0 | encoder | Sequential | 50.4 K | train
1 | decoder | Sequential | 51.2 K | train
-----------------------------------------------
101 K     Trainable params
0         Non-trainable params
101 K     Total params
0.407     Total estimated model params size (MB)
8         Modules in train mode
0         Modules in eval mode
C:\Users\José Jorge\miniconda3\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:433: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in

Training: |                                                                                      | 0/? [00:00<…

`Trainer.fit` stopped: `max_epochs=1` reached.


In [13]:
#import torch
import torch

# load checkpoint
checkpoint = "./lightning_logs/version_0/checkpoints/epoch=0-step=100.ckpt"
autoencoder = LitAutoEncoder.load_from_checkpoint(checkpoint, encoder=encoder, decoder=decoder) #El autoenconder iniciará a partir del entrenamiento

# choose your trained nn.Module
encoder = autoencoder.encoder #Solo nos centramos en el compresor
encoder.eval() #Lo pondremos en modo evaluación

# embed 4 fake images!
fake_image_batch = torch.rand(4, 28 * 28, device=autoencoder.device)
embeddings = encoder(fake_image_batch)
print("⚡" * 20, "\nPredictions (4 image embeddings):\n", embeddings, "\n", "⚡" * 20)

⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡ 
Predictions (4 image embeddings):
 tensor([[ 0.1067, -0.6335,  0.4315],
        [ 0.0869, -0.4465,  0.3336],
        [ 0.0603, -0.5975,  0.3820],
        [ 0.0376, -0.6302,  0.2050]], grad_fn=<AddmmBackward0>) 
 ⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡


BASIC SKILLS

Train a model (basic) Esto en realidad es muy similar a lo que ya se hizo.

LEVEL UP

Definimos los modulos de PyTorch (nn Modulos)

In [17]:
class Encoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Sequential(nn.Linear(28 * 28, 64), nn.ReLU(), nn.Linear(64, 3)) #Compresión de la imagen

    def forward(self, x): #Lo que hay que hacer con las entradas
        return self.l1(x)


class Decoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Sequential(nn.Linear(3, 64), nn.ReLU(), nn.Linear(64, 28 * 28)) #Descompresión

    def forward(self, x):
        return self.l1(x)

In [43]:
class LitAutoEncoder(L.LightningModule): #Este deberia ser un modulo completo para realizar el entrenamiento
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder #Este modulo lo que estaría haciendo es unir el compresor y el descompresor que configuramos anteriormente
        self.decoder = decoder #Aquí hacemos entonces, la conexión con los nn Modules

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        x, _ = batch
        x = x.view(x.size(0), -1) #Porqué es necesario aplastar la imagen???
        z = self.encoder(x) #Codificamos 
        x_hat = self.decoder(z) #Decodificamos
        loss = F.mse_loss(x_hat, x) #Calcula función de perdida que se tuvo en la reconstrucción
        return loss #Retorna la perdida 
    def test_step(self, batch, batch_idx):
        x, _ = batch
        x = x.view(x.size(0), -1) #Porqué es necesario aplastar la imagen???
        z = self.encoder(x) #Codificamos 
        x_hat = self.decoder(z) #Decodificamos
        test_loss = F.mse_loss(x_hat, x) #Calcula función de perdida que se tuvo en la reconstrucción
        self.log("test_loss", test_loss) #Guardar la evaluación de la pérdida

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3) #Aquí el optimizador ajusta los pesos del modelo 
        return optimizer #Regresa el optimizador 

La base de datos del entrenamiento:

In [54]:
train_set = MNIST(root="MNIST", download=True, train=True, transform=ToTensor())
train_set = utils.data.DataLoader(train_set)
test_set = MNIST(root="MNIST", download=True, train=False, transform=ToTensor())
test_set = utils.data.DataLoader(test_set)

Ahora sí, el entrenamiento:

In [55]:
# model
autoencoder = LitAutoEncoder(Encoder(), Decoder()) #Todo el modelo funcional completo

# train model
trainer = L.Trainer(limit_train_batches=10000, max_epochs=1) #Nuestro entrenador
trainer.fit(model=autoencoder, train_dataloaders=train_set)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name    | Type    | Params | Mode 
--------------------------------------------
0 | encoder | Encoder | 50.4 K | train
1 | decoder | Decoder | 51.2 K | train
--------------------------------------------
101 K     Trainable params
0         Non-trainable params
101 K     Total params
0.407     Total estimated model params size (MB)
10        Modules in train mode
0         Modules in eval mode


Training: |                                                                                      | 0/? [00:00<…

`Trainer.fit` stopped: `max_epochs=1` reached.


VALIDATE AND TEST A MODEL

In [45]:
# test the model
trainer = L.Trainer() #Nuestro entrenador
trainer.test(autoencoder, test_set)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
C:\Users\José Jorge\miniconda3\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:433: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.


Testing: |                                                                                       | 0/? [00:00<…

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.04559813067317009    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.04559813067317009}]

In [ ]:
ADD A VALIDATION LOOP

In [66]:
import torch.utils.data as data

# use 20% of training data for validation
train_set_size = int(len(train_set) * 0.8)
valid_set_size = len(train_set) - train_set_size

# split the train set into two
seed = torch.Generator().manual_seed(42)
train_set, valid_set = data.random_split(train_set, [train_set_size, valid_set_size], generator=seed)

Agregamos este paso al modelo:

In [48]:
class LitAutoEncoder(L.LightningModule): #Este deberia ser un modulo completo para realizar el entrenamiento
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder #Este modulo lo que estaría haciendo es unir el compresor y el descompresor que configuramos anteriormente
        self.decoder = decoder #Aquí hacemos entonces, la conexión con los nn Modules

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        x, _ = batch
        x = x.view(x.size(0), -1) #Porqué es necesario aplastar la imagen???
        z = self.encoder(x) #Codificamos 
        x_hat = self.decoder(z) #Decodificamos
        loss = F.mse_loss(x_hat, x) #Calcula función de perdida que se tuvo en la reconstrucción
        return loss #Retorna la perdida 
    def test_step(self, batch, batch_idx):
        x, _ = batch
        x = x.view(x.size(0), -1) #Porqué es necesario aplastar la imagen???
        z = self.encoder(x) #Codificamos 
        x_hat = self.decoder(z) #Decodificamos
        test_loss = F.mse_loss(x_hat, x) #Calcula función de perdida que se tuvo en la reconstrucción
        self.log("test_loss", test_loss) #Guardar la evaluación de la pérdida
    
    def validation_step(self, batch, batch_idx):
        # this is the validation loop
        x, _ = batch
        x = x.view(x.size(0), -1)
        z = self.encoder(x)
        x_hat = self.decoder(z)
        val_loss = F.mse_loss(x_hat, x)
        self.log("val_loss", val_loss)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3) #Aquí el optimizador ajusta los pesos del modelo 
        return optimizer #Regresa el optimizador 

ESTA CELDA YA NO LA PUDÉ CORRER :(((

In [70]:

train_loader = utils.data.DataLoader(train_set)
valid_loader = utils.data.DataLoader(valid_set)

# train with both splits
trainer = L.Trainer()
trainer.fit(
    model=autoencoder,
    train_dataloaders=train_loader)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name    | Type    | Params | Mode
-------------------------------------------
0 | encoder | Encoder | 50.4 K | eval
1 | decoder | Decoder | 51.2 K | eval
-------------------------------------------
101 K     Trainable params
0         Non-trainable params
101 K     Total params
0.407     Total estimated model params size (MB)
0         Modules in train mode
10        Modules in eval mode


Training: |                                                                                      | 0/? [00:00<…

TypeError: 'DataLoader' object is not subscriptable

ABOUT CHECKPOINTS

In [60]:
# simply by using the Trainer you get automatic checkpointing
trainer = L.Trainer()

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [62]:
# saves checkpoints to 'some/path/' at every epoch end
trainer = L.Trainer(default_root_dir="some/path/")

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


EARLY STOPPING

Para detener cuando un parametro deje de mejorar:

In [ ]:
from lightning.pytorch.callbacks.early_stopping import EarlyStopping


class LitAutoEncoder(L.LightningModule): #Este deberia ser un modulo completo para realizar el entrenamiento
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder #Este modulo lo que estaría haciendo es unir el compresor y el descompresor que configuramos anteriormente
        self.decoder = decoder #Aquí hacemos entonces, la conexión con los nn Modules

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        x, _ = batch
        x = x.view(x.size(0), -1) #Porqué es necesario aplastar la imagen???
        z = self.encoder(x) #Codificamos 
        x_hat = self.decoder(z) #Decodificamos
        loss = F.mse_loss(x_hat, x) #Calcula función de perdida que se tuvo en la reconstrucción
        return loss #Retorna la perdida 
    def validation_step(self, batch, batch_idx):
        x, _ = batch
        x = x.view(x.size(0), -1) #Porqué es necesario aplastar la imagen???
        z = self.encoder(x) #Codificamos 
        x_hat = self.decoder(z) #Decodificamos
        loss = F.mse_loss(x_hat, x) #Calcula función de perdida que se tuvo en la reconstrucción
        self.log("val_loss", loss)
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3) #Aquí el optimizador ajusta los pesos del modelo 
        return optimizer #Regresa el optimizador 


model = LitModel()
trainer = Trainer(callbacks=[EarlyStopping(monitor="val_loss", mode="min")])
trainer.fit(model)